In [1]:
# Import
import pandas as pd
import numpy as np
pd.set_option('display.float_format', '{:.2f}'.format)

In [2]:
Avversari = "FOSSOMBRONE"
Squadra   = "URBANIA"

In [3]:
# Lettura dei risultati
dfRisultati = pd.read_csv('Risultati.csv')

# Eliminazione righe nulle
dfRisultati = dfRisultati[dfRisultati['Squadra'].notnull()]

# Cambio dei nomi delle colonne
columns = dfRisultati.columns
newnames = []
for index, c in enumerate(columns):
    if c[:8] != 'Unnamed:':
        newnames.append(c)
        
newnames = newnames[1:]
newnames = ['Squadra'] + [x+ndx for x in newnames for ndx in ('', '2','3')]

dfRisultati.columns = newnames
#dfRisultati.to_csv('TabellaRisultati.csv')
#display(dfRisultati)

In [4]:
# Nomi delle squadre
squadre = dfRisultati['Squadra'].unique()

# Estrazione delle statistiche di squadra ad una determinata giornata
def EstraiStats(finoallagiornata=999):
    # Raccolta informazioni su ciascuna squadra
    partite_totali  = []
    partite_casa    = []
    partite_fuori   = []
    vittorie_totali = []
    vittorie_casa   = []
    vittorie_fuori  = []
    difcan_totali   = []
    difcan_casa     = []
    difcan_fuori    = []
    segnati_totali  = []
    segnati_casa    = []
    segnati_fuori   = []
    subiti_totali   = []
    subiti_casa     = []
    subiti_fuori    = []

    for s in squadre:
        pt = 0
        pc = 0
        pf = 0
        vt = 0
        vc = 0
        vf = 0
        dt = 0
        dc = 0
        df = 0
        st = 0
        sc = 0
        sf = 0
        ut = 0
        uc = 0
        uf = 0

        # Ciclo sulle partite in casa
        dfs = dfRisultati[dfRisultati['Squadra']==s]
        for a in squadre:
            dfcasa = dfs[dfs[a].notnull()][[a, a+'2',a+'3']]
            if dfcasa.shape[0] > 0:
                punteggio = list(dfcasa.iloc[0])
                giornata = punteggio[0]
                if giornata <= finoallagiornata:
                    pt += 1
                    pc += 1
                    ps = punteggio[1]
                    pa = punteggio[2]
                    st += ps
                    sc += ps
                    ut += pa
                    uc += pa
                    dt += ps - pa
                    dc += ps - pa
                    if ps > pa:
                        vt += 1
                        vc += 1

        # Ciclo sulle partite in trasferta
        dfs = dfRisultati[dfRisultati[s].notnull()][['Squadra',s,s+'2',s+'3']]
        for index, row in dfs.iterrows():
            giornata = row[s]
            if giornata <= finoallagiornata:
                a = row['Squadra']
                pa = row[s+'2']
                ps = row[s+'3']
                st += ps
                sf += ps
                ut += pa
                uf += pa
                pt += 1
                pf += 1
                dt += ps - pa
                df += ps - pa
                if ps > pa:
                    vt += 1
                    vf += 1

        partite_totali.append(pt)
        partite_casa.append(pc)
        partite_fuori.append(pf)
        vittorie_totali.append(vt)
        vittorie_casa.append(vc)
        vittorie_fuori.append(vf)
        difcan_totali.append(dt)
        difcan_casa.append(dc)
        difcan_fuori.append(df)
        segnati_totali.append(st)
        segnati_casa.append(sc)
        segnati_fuori.append(sf)
        subiti_totali.append(ut)
        subiti_casa.append(uc)
        subiti_fuori.append(uf)
        
    # Creazione del DataFrame delle statistiche di squadra
    dfStats = pd.DataFrame({'Squadra'       : squadre,
                            'PartiteTotali' : partite_totali,
                            'PartiteCasa'   : partite_casa,
                            'PartiteFuori'  : partite_fuori,
                            'VittorieTotali': vittorie_totali,
                            'VittorieCasa'  : vittorie_casa,
                            'VittorieFuori' : vittorie_fuori,
                            'DifCanTotali'  : difcan_totali,
                            'DifCanCasa'    : difcan_casa,
                            'DifCanFuori'   : difcan_fuori,
                            'SegnatiTotali' : segnati_totali,
                            'SegnatiCasa'   : segnati_casa,
                            'SegnatiFuori'  : segnati_fuori,
                            'SubitiTotali'  : subiti_totali,
                            'SubitiCasa'    : subiti_casa,
                            'SubitiFuori'   : subiti_fuori,
                            'Punti'         : [x*2 for x in vittorie_totali]
                          })

    # Conversione di tutte le colonne numeriche ad intere
    m = dfStats.select_dtypes(np.number)
    dfStats[m.columns] = m.round().astype('Int64')
    
    # Medie segnati e subiti
    dfStats['MediaSegnatiTotali'] = dfStats['SegnatiTotali'] / dfStats['PartiteTotali']
    dfStats['MediaSegnatiCasa']   = dfStats['SegnatiCasa']   / dfStats['PartiteCasa']
    dfStats['MediaSegnatiFuori']  = dfStats['SegnatiFuori']  / dfStats['PartiteFuori']
    
    dfStats['MediaSubitiTotali'] = dfStats['SubitiTotali'] / dfStats['PartiteTotali']
    dfStats['MediaSubitiCasa']   = dfStats['SubitiCasa']   / dfStats['PartiteCasa']
    dfStats['MediaSubitiFuori']  = dfStats['SubitiFuori']  / dfStats['PartiteFuori']

    # Differenza Canestri media
    dfStats['MediaDifCanTotali'] = dfStats['DifCanTotali'] / dfStats['PartiteTotali']
    dfStats['MediaDifCanCasa']   = dfStats['DifCanCasa']   / dfStats['PartiteCasa']
    dfStats['MediaDifCanFuori']  = dfStats['DifCanFuori']  / dfStats['PartiteFuori']
    
    # Percentuale vittorie per turno
    dfStats['PercVittorieTotali'] = 100.0 * dfStats['VittorieTotali'] / dfStats['PartiteTotali']
    dfStats['PercVittorieCasa']   = 100.0 * dfStats['VittorieCasa']   / dfStats['PartiteCasa']
    dfStats['PercVittorieFuori']  = 100.0 * dfStats['VittorieFuori']  / dfStats['PartiteFuori']
    
    # Ordinamento per classifica
    dfStats = dfStats.sort_values(['Punti', 'DifCanTotali'], ascending=[False, False])
    
    # Index su squadra
    dfStats = dfStats.reset_index().set_index('Squadra')

    # Index sulla posizione partendo da 1
    dfStats = dfStats.reset_index()
    dfStats.index += 1 
    
    return dfStats

In [5]:
# Classifica attuale nel formato BasketMarche
dfStats = EstraiStats()
dfClassifica = dfStats[['Squadra','Punti','PartiteTotali','VittorieTotali', 'SegnatiTotali', 'SubitiTotali', 'DifCanTotali', 'MediaSegnatiTotali', 'MediaSubitiTotali']]
dfClassifica = dfClassifica.rename(columns = {'Punti':'Pt', 'PartiteTotali': 'G', 'VittorieTotali': 'V',
                                              'SegnatiTotali': 'Pse', 'SubitiTotali': 'Psu', 'DifCanTotali': 'D',
                                              'MediaSegnatiTotali': 'Mse', 'MediaSubitiTotali': 'Msu'})
dfClassifica['P'] = dfClassifica['G'] - dfClassifica['V']
dfClassifica = dfClassifica[['Squadra','Pt','G','V','P','Pse','Psu','Mse','Msu','D']]
dfClassifica.index.name = 'Posizione'
#dfClassifica.to_csv('Classifica.csv')
#dfClassifica

In [6]:
# Calcola la difficoltà nell'affrontare una squadra basandosi sulla posizione corrente in classifica.
# Torna un valore da 1 (ultima in classifica) a 15, prima in classifica
def DifficoltaSquadra(squadra):
    posizione = list(dfClassifica[dfClassifica['Squadra']==squadra].index)[0]
    return len(squadre) + 1 - posizione


# Estrae dai risultati tutte le partite disputate da una squadra
def TutteLePartite(squadra):
    partite = []
    # Ciclo sulle partite in casa
    dfs = dfRisultati[dfRisultati['Squadra']==squadra]
    for a in squadre:
        dfcasa = dfs[dfs[a].notnull()][[a, a+'2',a+'3']]
        if dfcasa.shape[0] > 0:
            punteggio = list(dfcasa.iloc[0])
            giornata       = int(punteggio[0])
            puntisquadra   = int(punteggio[1])
            puntiavversari = int(punteggio[2])
            difficolta = DifficoltaSquadra(a)
            partite.append((squadra,True,giornata,a,puntisquadra,puntiavversari,difficolta))
            
    # Ciclo sulle partite in trasferta
    dfs = dfRisultati[dfRisultati[squadra].notnull()][['Squadra',squadra,squadra+'2',squadra+'3']]
    for index, row in dfs.iterrows():
        avversario = row['Squadra']
        giornata       = int(row[squadra])
        puntisquadra   = int(row[squadra+'3'])
        puntiavversari = int(row[squadra+'2'])
        difficolta = DifficoltaSquadra(avversario) * 1.25    # Difficoltà in trasferta aumentata del 25%
        partite.append((squadra,False,giornata,avversario,puntisquadra,puntiavversari,difficolta))
            
    return sorted(partite,  key=lambda x: x[2])


elenco_partite = {}
for s in squadre:
    elenco_partite[s] = TutteLePartite(s)

    
# Torna la difficolta media delle partite affrontate da una squadra fino ad una determinata giornata    
def difficoltaMedia(squadra, giornata):
    partite = [x for x in elenco_partite[squadra] if x[2] <= giornata]
    casa      = [x[6] for x in partite if x[1]]
    trasferta = [x[6] for x in partite if not x[1]]
    if len(partite) > 0:
        return (sum(casa) + sum(trasferta))/float(len(partite))
    else:
        return 0


# Estrazione dei dati su tutte le giornate disputate fino ad ora

# Numero giornate disputate
numerogiornate = int(dfRisultati[squadre].fillna(0).to_numpy().max())

dfStats = None
for n in range(1,numerogiornate+1):
    dfStatsG = EstraiStats(n)
    dfStatsG.columns = ['Squadra'] + [x+str(n) for x in dfStatsG.columns if x != 'Squadra']
    dfStatsG = dfStatsG.drop('index'+str(n), axis=1)
    
    dfStatsG['Difficolta'+str(n)] = [difficoltaMedia(s,n) for s in dfStatsG['Squadra']]
        
    if dfStats is None:
        dfStats = dfStatsG
    else:
        dfStats = pd.merge(dfStats, dfStatsG,  how='left', left_on=['Squadra'], right_on=['Squadra'])


# Ordinamento per classifica
dfStats = dfStats.sort_values(['Punti'+str(numerogiornate), 'DifCanTotali'+str(numerogiornate)], ascending=[False, False])

# Index sulla posizione partendo da 1
dfStats = dfStats.reset_index()
dfStats.index += 1 

dfStats = dfStats.drop('index', axis=1)
#dfStats

In [7]:
import plotly.express as px
import plotly.graph_objects as go

In [8]:
def EstraiTimeline(dfStats, numerogiornate, nomecampo):
    # Lista delle colonne dei punti di tutte le giornate
    colonne = [nomecampo+str(x+1) for x in range(numerogiornate)]
    renamed = [str(x+1) for x in range(numerogiornate)]

    # Evoluzione dei punti nelle giornate  --> Grafico
    return dfStats[['Squadra']+colonne].rename(columns = dict(zip(colonne,renamed)))
    

#dfTimeline = EstraiTimeline(dfStats, numerogiornate, 'Punti')
#dfTimeline = EstraiTimeline(dfStats, numerogiornate, 'DifCanTotali')
#dfTimeline = EstraiTimeline(dfStats, numerogiornate, 'MediaSubitiTotali')
#dfTimeline = EstraiTimeline(dfStats, numerogiornate, 'PercVittorieTotali')
#dfTimeline = dfTimeline.fillna(0.0)  # Non funziona!
#dfTimeline = dfTimeline.replace(np.nan, 0)  # Non funziona!
#dfTimeline

In [9]:
#list(dfStats.columns)

In [10]:
# Aggiunge un line chart per una squadra
def addTrace(fig, dfTimeline, squadra, color='red', textpos='top center'):
    df = dfTimeline[dfTimeline['Squadra']==squadra]
    df.set_index('Squadra')
    fig.add_traces(go.Scatter(x=df.columns[1:],
                              y=df.values.flatten().tolist()[1:], 
                              text=['%.1f'%x for x in df.values.flatten().tolist()[1:]],
                              mode='lines+markers+text',
                              textposition=textpos,
                              line=dict(color=color),
                              name=squadra))
    
def creaGrafico(nomecampo, titolo):
    dfTimeline = EstraiTimeline(dfStats, numerogiornate, nomecampo)
    
    dfSquadra   = dfTimeline[dfTimeline['Squadra']==Squadra]
    dfAvversari = dfTimeline[dfTimeline['Squadra']==Avversari]
    valuesSquadra   = dfSquadra.values.flatten().tolist()[1:]
    valuesAvversari = dfAvversari.values.flatten().tolist()[1:]
    valuesSquadra   = [0 if pd.isna(x) else x for x in valuesSquadra]
    valuesAvversari = [0 if pd.isna(x) else x for x in valuesAvversari]
    if sum(valuesSquadra) > sum(valuesAvversari):
        textposSquadra   = 'bottom center'
        textposAvversari = 'top center'
    else:
        textposAvversari = 'bottom center'
        textposSquadra   = 'top center'
    
    fig = go.Figure()
    addTrace(fig,dfTimeline, Squadra,   'red',  textposSquadra)
    addTrace(fig,dfTimeline, Avversari, 'blue', textposAvversari)
    fig.update_layout(margin=dict(l=20, r=20, t=50, b=20),
                      height=300,
                      title={'text': '<b>' + titolo + '</b>', 'y':0.95, 'x':0.0, 'xanchor': 'left', 'yanchor': 'top'},
                      template='plotly_white',
                      xaxis_title="Giornate",)
    #fig.show(config={'displayModeBar': False})
    #display(HTML('</br></br></br></br>'))
    return fig.to_image(format="png")

#reaGrafico('PercVittorieFuori',  'Percentuale di vittorie fuori casa')

In [11]:
# Generazione dei grafici in formato PNG
png1 = creaGrafico('PercVittorieTotali', 'Percentuale totale di vittorie')
png2 = creaGrafico('PercVittorieCasa',   'Percentuale di vittorie in casa')
png3 = creaGrafico('PercVittorieFuori',  'Percentuale di vittorie fuori casa')
png4 = creaGrafico('MediaSegnatiTotali', 'Media punti segnati totale')
png5 = creaGrafico('MediaSegnatiCasa',   'Media punti segnati in casa')
png6 = creaGrafico('MediaSegnatiFuori',  'Media punti segnati fuori casa')
png7 = creaGrafico('MediaSubitiTotali',  'Media punti subiti totale')
png8 = creaGrafico('MediaSubitiCasa',    'Media punti subiti in casa')
png9 = creaGrafico('MediaSubitiFuori',   'Media punti subiti fuori casa')

In [12]:
# Display di una lista di partite di una squadra su un documento docx
def displayPartiteInDocx(document, partite, titolo=''):
    ncasa  = len([x for x in partite if x[1]])
    nfuori = len([x for x in partite if not x[1]])
    totdifficolta = 0
    for p in partite:
        squadra,incasa,giornata,avversario,puntisquadra,puntiavversario,difficolta = p
        totdifficolta += difficolta

    table = document.add_table(rows=len(partite)+1, cols=6)
    table.cell(0,0).text = 'Giornata'
    table.cell(0,0).paragraphs[0].style = document.styles['Bold']
    
    table.cell(0,1).text = titolo
    table.cell(0,1).paragraphs[0].style = document.styles['Bold']

    table.cell(0,2).text = '(%dC %dF)' % (ncasa,nfuori)
    table.cell(0,2).paragraphs[0].style = document.styles['Bold']

    table.cell(0,3).text = 'Risultato'
    table.cell(0,3).paragraphs[0].style = document.styles['Bold']

    table.cell(0,4).text = 'Scarto'
    table.cell(0,4).paragraphs[0].style = document.styles['Bold']
    
    table.cell(0,5).text = 'D: %.2f' % totdifficolta
    table.cell(0,5).paragraphs[0].style = document.styles['Bold']
    #table.cell(0,5).paragraphs[0].paragraph_format.alignment = WD_ALIGN_PARAGRAPH.CENTER

    table.cell(0,0).width = Inches(0.5)
    table.cell(0,1).width = Inches(1.7)
    table.cell(0,2).width = Inches(1.7)
    table.cell(0,3).width = Inches(0.4)
    table.cell(0,4).width = Inches(0.4)
    table.cell(0,4).width = Inches(1.4)
    
    for index,p in enumerate(partite):
        squadra,incasa,giornata,avversario,puntisquadra,puntiavversario,difficolta = p
        vinta = puntisquadra > puntiavversario
        if vinta:
            style = 'Green'
        else:
            style = 'Red'

        table.cell(index+1,0).text = '%d.a' % giornata
        
        if incasa:
            table.cell(index+1,1).text = squadra
            table.cell(index+1,1).paragraphs[0].style = document.styles[style]

            table.cell(index+1,2).text = '- ' + avversario
            table.cell(index+1,2).paragraphs[0].style = document.styles[style]
            
            table.cell(index+1,3).text = '%d-%d' % (puntisquadra,puntiavversario)
        else:
            table.cell(index+1,1).text = avversario
            table.cell(index+1,1).paragraphs[0].style = document.styles[style]

            table.cell(index+1,2).text = '- ' + squadra
            table.cell(index+1,2).paragraphs[0].style = document.styles[style]
            
            table.cell(index+1,3).text = '%d-%d' % (puntiavversario,puntisquadra)
            
        if vinta: table.cell(index+1,4).text = '+ %d' % abs(puntisquadra-puntiavversario)
        else:     table.cell(index+1,4).text = '-  %d' % abs(puntisquadra-puntiavversario)

        table.cell(index+1,5).text = '%.2f' % difficolta
        #table.cell(index+1,5).paragraphs[0].paragraph_format.alignment = WD_ALIGN_PARAGRAPH.CENTER
            
    for row in table.rows:
        row.height_rule = WD_ROW_HEIGHT_RULE.EXACTLY
        row.height = Inches(0.21)

In [13]:
# Creazione del report in formato DOCX
from docx import Document
from docx.shared import Inches
from docx.enum.text import WD_ALIGN_PARAGRAPH
from docx.enum.table import WD_ROW_HEIGHT_RULE
from docx.oxml.ns import nsdecls
from docx.oxml import parse_xml

from ipywidgets import widgets, Layout
import io

out = widgets.Output(layout=Layout(width='0px', height='0px'))
display(out)


# Direct download of an array of bytes
def downloadBytes(bytesobj, fileName="download.bin"):
    import base64
    from IPython.display import display, HTML
    
    base64_bytes  = base64.b64encode(bytesobj)
    base64_string = base64_bytes.decode("ascii")

    out.clear_output()
    with out:
        display(HTML('''
<script>function downloadURI(uri, name)
{
    var link = document.createElement("a");
    link.download = name;
    link.href = uri;
    link.click();
}
downloadURI("data:application/octet-stream;charset=utf-8;base64,''' + base64_string + '","' + fileName + '");</script>'))
        
    out.clear_output()

    
# Creation of report in .docx format using a template
document = Document('AnalisiAvversari.docx')
document.add_paragraph('Squadra avversaria: ' + Avversari, style='Bold')
document.add_paragraph('')

#for s in document.styles:
#    print(s.name)

imageswidth = 17 / 2.54

# Tabella per le partite della nostra squadra e degli avversari
displayPartiteInDocx(document, TutteLePartite(Squadra),Squadra)
document.add_paragraph('')
displayPartiteInDocx(document, TutteLePartite(Avversari),Avversari)

document.add_page_break()
document.add_paragraph('')
document.add_paragraph('CLASSIFICA: ', style='Bold')

# Tabella per la classifica del campionato
table = document.add_table(rows=dfClassifica.shape[0]+1, cols=11)
table.cell(0,0).text = 'Posizione'
table.cell(0,0).paragraphs[0].style = document.styles['Bold']
for index,c in enumerate(dfClassifica.columns):
    table.cell(0,index+1).text = c
    
    table.cell(0,index+1).paragraphs[0].style = document.styles['Bold']
    
    # Differenza canestri centrata
    if index == len(dfClassifica.columns)-1:
        table.cell(0,index+1).paragraphs[0].paragraph_format.alignment = WD_ALIGN_PARAGRAPH.CENTER

table.cell(0,1).width = Inches(2.0)

for i in range(dfClassifica.shape[0]):
    table.cell(i+1,0).text = str(i+1)
    table.cell(i+1,0).paragraphs[0].paragraph_format.alignment = WD_ALIGN_PARAGRAPH.CENTER
    
    for index,c in enumerate(dfClassifica.columns):
        value = dfClassifica.loc[i+1][index]
        if isinstance(value, float):
            value = "{:.2f}".format(value)
            
        table.cell(i+1,index+1).text = str(value)
        
        if index == 0:
            table.cell(i+1,index+1).paragraphs[0].style = document.styles['Bold']

        # Differenza canestri centrata
        if index == len(dfClassifica.columns)-1:
            table.cell(i+1,index+1).paragraphs[0].paragraph_format.alignment = WD_ALIGN_PARAGRAPH.CENTER


for row in table.rows:
    row.height_rule = WD_ROW_HEIGHT_RULE.EXACTLY
    row.height = Inches(0.21)

document.add_paragraph('')


document.add_page_break()
document.add_paragraph('')
document.add_picture(io.BytesIO(png1), width=Inches(imageswidth))
document.add_paragraph('')
document.add_picture(io.BytesIO(png2), width=Inches(imageswidth))
document.add_paragraph('')
document.add_picture(io.BytesIO(png3), width=Inches(imageswidth))

document.add_paragraph('')
document.add_picture(io.BytesIO(png4), width=Inches(imageswidth))
document.add_paragraph('')
document.add_picture(io.BytesIO(png5), width=Inches(imageswidth))
document.add_paragraph('')
document.add_picture(io.BytesIO(png6), width=Inches(imageswidth))

document.add_paragraph('')
document.add_picture(io.BytesIO(png7), width=Inches(imageswidth))
document.add_paragraph('')
document.add_picture(io.BytesIO(png8), width=Inches(imageswidth))
document.add_paragraph('')
document.add_picture(io.BytesIO(png9), width=Inches(imageswidth))

buf = io.BufferedRandom(io.BytesIO())
document.save(buf)
buf.seek(0)
barray = buf.read()
downloadBytes(barray,"Analisi_%s.docx" % Avversari)

Output(layout=Layout(height='0px', width='0px'))